In [13]:
def read_content(path, threshold = False):
    '''
    threshold: if is True, it's represent document that filter the first between third lines,
               because it's noise.
    result: cleaned content
    '''
    #Step 1.1 open document or query
    content = open(path, 'r')
    #Step 1.2 read content line by line
    lines = content.read().splitlines()
    #Step 1.3 decide is document or query
    if threshold is True:
        content = lines[3:]
    else:
        content = lines
    #Step 1.4 added all of lines in result
    result = ''
    for i in range(len(content)):
        result = result + content[i].strip('-1')    
    return result

In [14]:
def word_lexicon(N, document): 
    '''
    lexicon: is word dictionary which including all of document and it stored index.
    N: docu_clean length
    '''
    #Step 3.1 create lexicon dictionary and index
    lexicon = {}
    count = 0
    #Step 3.2 decide if word not in the lexicon then update lexicon key and stored index
    for i in range(N):
        if document[i] not in lexicon:
            lexicon.update({document[i]: count})
            count += 1
    return lexicon

In [79]:
def tf_frequency(document, lexicon):
    '''
    tf: term frequency(row is document length, column is lexicon length)
    lexicon: word dictionary
    document: all document or query
    '''
    #Step 4.1 create tf 2d-array
    tf = np.zeros((len(document), len(lexicon)))
    #Step 4.2 judged tf shape
    assert tf.shape == (len(document), len(lexicon)), 'tf shape error'
    #Step 4.3 map each documents
    for n in range(len(document)):
        #Step 4.4 map each words of document
        for word in document[n].split():
            #Step 4.5 determine which word in lexicon existed
            if (word in lexicon):
                #finded this word index and in tf[this document][this word index] + 1
                idx = lexicon[word]
                tf[n][idx] += 1
    return tf

In [89]:
def idf_transform(N, tf):
    '''
    N: lexicon length
    tf: document tf
    '''
    #Step 5.1 create tf 1d-vector that type is float
    idf = np.zeros(N, dtype = float)
    #Step 5.2 judge idf shape
    assert idf.shape[0] == tf.shape[1], 'idf shape error'
    #Step 5.3 map lexicon word
    for word in range(N):
        n = 0
        #Step 5.4 map each document words and calculate it's idf
        for d in range(tf.shape[0]):
            if (tf[d][word] > 0):
                n += 1
        #Step 5.5 calculate idf 
        idf[word] = np.log((N - n + 0.5) / (n + 0.5))
    return idf

In [90]:
def compute_tfij_(N, b, document, tf):
    #Step 6.1 used advanced that calculated each document length
    dl = [len(document[i].split()) for i in range(len(document))]
    #Step 6.2 computed average document
    avg_dl = np.divide(np.sum(dl), N)
    #Step 6.3 computed K
    K = (1 - b) + (b * (dl / avg_dl))
    #reshape K shape (1d-vector → 2d-array)
    K = np.reshape(K, (K.shape[0], 1))
    #Step 6.4 computed tfi,j'
    tfij_ = np.divide(tf, K)
    return tfij_

In [91]:
def bm25_score(k1, k3, **kwargs):
    '''
    query: query content
    document: document content
    lexicon: word dictionary
    tfij_: tfi,j'
    tf_query: tfi,q
    idf: idf
    '''
    #Step 7.1 receive dictionary type data 
    query = kwargs['query']
    document = kwargs['document']
    lexicon = kwargs['lexicon']
    tfij_ = kwargs['tfij_']
    tf_query = kwargs['tf_query']
    idf = kwargs['idf']
    #Step 7.2 create score 2d-array that row is query length, column is document length
    score = np.zeros((len(query),len(document)))
    #Step 7.3 judge score shape
    assert score.shape == (len(query), len(document)), 'score shape error'
    #Step 7.4 map each query
    for qlen in range(len(query)):
        #Step 7.5 map each documnet
        for dlen in range(len(document)):
            #Step 7.6 filter word in document and query appear simultaneously
            intersection = set(document[dlen].split()) & set(query[qlen].split())
            for word in intersection:
                idx = lexicon[word]
                #Step 7.7 computed ((k1 + 1) * tfi,j'[document][word index]) / (tfi,j'[document][word index] + k1)
                B = np.divide((k1 + 1) * tfij_[dlen][idx], (tfij_[dlen][idx] +  k1))
                #Step 7.8 computed ((k3 + 1) * tfi,q[query][word index])) / (k3 + tfi,q[query][word index])
                G = np.divide((k3 + 1) * tf_query[qlen][idx], (k3 + tf_query[qlen][idx]))
                #Step 7.9 computed B * G * idf[index] in score[query][document]
                score[qlen][dlen] += (B * G * idf[idx])
    return score

In [92]:
def score_sort(N, docu_files, score):
    '''
    N: query length
    docu_files: document filenames
    score: simularity in each document
    '''
    lis = {}
    for q in range(N):
        #Step 8.1 sorted each document score in this query
        a = np.argsort(-score[q])
        #Step 8.2 declare lis[q + 1] and content is null
        lis['%s'% (q + 1)] = []
        #Step 8.3 append to list
        for i in range(len(a)):
            lis['%s'% (q + 1)].append(docu_files[a[i]])
    return lis

In [93]:
def write_text(quer_files, document, lis):
    '''
    quer_files: query filenames
    document: document content
    lis: sorted result
    '''
    #Step 10.1 write into submission.txt
    with open('submission.txt', 'a', encoding = 'utf-8') as f:
        f.write('Query,RetrievedDocuments\n')
        for i in range(len(quer_files)):
            f.write(quer_files[i] + ',')
            for v in lis['%s' % (i + 1)]:
                f.write(v + ' ')
            f.write('\n')

In [120]:
import os
import sys
import re
import numpy as np
from collections import Counter

def main():
    #Step 0. Get document, query path and filenames
    document_path = os.path.abspath('Document')
    query_path = os.path.abspath('Query')
    docu_files = os.listdir(document_path)
    quer_files = os.listdir(query_path)
    
    #Step 1. Read document and query content
    document = []
    for i in range(len(docu_files)):
        document.append(read_content('Document\\' + docu_files[i], threshold = True))

    query = []
    for j in range(len(quer_files)):
        query.append(read_content('Query\\' + quer_files[j]))
    
    #Step 2. used regular expression to get words from all of document and query
    docu_clean = [re.sub(r"[^0-9]+", ' ', document[i].split()[j]) for i in range(len(document)) for j in range(len(document[i].split()))]
    
    #Step 3. build word dictionary
    lexicon = word_lexicon(len(docu_clean), docu_clean)
    #Step 4. computed term-frequency of document and query 
    tf = tf_frequency(document, lexicon)
    tf_query = tf_frequency(query, lexicon)
    #Step 5. computed intrieve-document-frequency of document
    idf = idf_transform(len(lexicon), tf)
    
    N = len(document)    
    k1 = 5
    k3 = 1000
    b = 0.75
    #Step 6. computed term-frequecny i,j'
    tfij_ = compute_tfij_(N, b, document, tf)
    #Step 7. computed each document simularity of each query
    score = bm25_score(k1, k3, query = query, document = document, lexicon = lexicon, tfij_ = tfij_,\
                       tf_query = tf_query, idf = idf)
    #Step 8. sotred the score
    sort_score = score_sort(len(query), docu_files, score)
    #Step 9. judge which is existed or not
    if os.path.exists('submission.txt'):
        os.remove('submission.txt')
    #Step 10. write sorted result into submission.txt
    write_text(quer_files, document, sort_score)

In [121]:
#Executing main function
if __name__ == '__main__':
    main()